# Introduction to programming - Assignment 2019/2020

**Submit to Blackboard before wednesday, 29th of April at 12:00 (noon).**

**6 marks attainable, corresponding to 40% of the course grade (provided the attendance criteria have been met)**

The moment of inertia is an important structural feature of molecules. It not only defines all the rotational properties of a molecule and their spectroscopic manifestations, but it also provides a generic way of characterising molecular shape.

![You should see an image of a prolate and an oblate molecule. Check if the notebook is being run on the same folder as the accompanying files.](symmetric_tops.svg "Adapted from J.L.McHale «Molecular Spectroscopy» 2nd ed. CRC press 2017")

The goal of this exercise is to write a standalone python program, called *moi.py*, which should ask the user for the name of <a href="http://openbabel.org/wiki/XYZ_(format)#Additional_Comments">XYZ file</a>, after which the program should print the value of the moment of inertia along the 3 principal axis of rotation of the molecule, in the format:

    -- Ia / Ib / Ic

where Ia, Ib and Ic are the numerical values of the moment of inertia in units g.mol<sup>-1</sup>.&Aring;<sup>2</sup> (within a tolerance of 1&times;10<sup>-3</sup> g.mol<sup>-1</sup>.&Aring;<sup>2</sup>). No particular order is needed, but it is important that the line starts with the characters '--' and the values are separated by a forward dash '/', as this will be used to test the output of your program. Your program can output any text you wish above or below your result (as long as it does not start with '--').

All molecules possess 3 principal axis of rotation which are orthogonal and intercept at the position of the centre of mass. The moment of inertia is measured with respect to these 3 axis.

<img src="principal_axis.png" style="width:200px" alt="You should see an image of a molecule and its principal rotational axis. Check if the notebook is being run on the same folder as the accompanying files." title="Adapted CC BY-NC-SA material from J.Simons «Advanced Theoretical Chemistry (Simons)» LibreTexts 2019 - https://chem.libretexts.org/Bookshelves/Physical_and_Theoretical_Chemistry_Textbook_Maps/Book%3A_Advanced_Theoretical_Chemistry_(Simons)/5%3A_An_Overview_of_Theoretical_Chemistry/5.2%3A_Molecular_Structure%3A_Theory_and_Experiment" />

In order to calculate the 3 moment of inertia of an arbitrary molecular structure in a XYZ file, there are two things one needs to consider:
1. The centre of mass of the molecule may not coincide with the origin of the reference frame.
2. The principal axes of rotation, around which I<sub>a</sub>, I<sub>b</sub> and I<sub>c</sub> are measured, may not be aligned with the cartesian reference frame axes.

To deal with issue 1, one needs first to calculate the position of the centre of mass $\underline{r_{CM}}$

$$\underline{r_{CM}}=\frac{1}{m_{total}} \sum_{\alpha} m_{\alpha} \underline{r_{\alpha}}$$

where the sum runs over all atoms in the molecule, $m_{\alpha}$ and $\underline{r_{\alpha}}$ are the mass and the position of atom &alpha;, and $m_{total}$ in the total mass of the molecule. Once the position of the centre of mass is know, one subtracts $\underline{r_{CM}}$ to the position of all atoms to obtain a new set of coordinates $\underline{r^{CM}_{\alpha}}$ which have $\underline{r_{CM}}$ as the origin.

![You should see an image of a molecule on the original frame of reference and the same molecule on the centre of mass reference frame. Check if the notebook is being run on the same folder as the accompanying files.](CM_removal.svg)

To calculate the 3 moment of inertia associated with the principal axes of rotation we first build the 3&times;3 inertia matrix (or inertia tensor) $\underline{\underline{I}}$ (not to be confused with the identity matrix):

$$\underline{\underline{I}}=\left[ \begin{array}{ccc} I_{xx} & I_{xy} & I_{xz} \\ I_{yx} & I_{yy} & I_{yz} \\ I_{zx} & I_{zy} & I_{zz} \end{array}\right]$$

where

$$I_{xx}= \sum_{\alpha} m_{\alpha} (y_{\alpha}^2+z_{\alpha}^2)$$
$$I_{yy}= \sum_{\alpha} m_{\alpha} (x_{\alpha}^2+z_{\alpha}^2)$$
$$I_{zz}= \sum_{\alpha} m_{\alpha} (x_{\alpha}^2+y_{\alpha}^2)$$

and $x_{\alpha}$, $y_{\alpha}$ and $z_{\alpha}$ are the coordinates of atom &alpha; in the center of mass reference. The term $(y_{\alpha}^2+z_{\alpha}^2)$ corresponds to the distance of atom &alpha; to the *x* axis (and similarly for other axis). The off-diagonal terms take the general form

$$I_{xy}=-\sum_{\alpha} m_{\alpha} x_{\alpha} y_{\alpha}$$
$$I_{xz}=-\sum_{\alpha} m_{\alpha} x_{\alpha} z_{\alpha}$$
$$I_{yz}=-\sum_{\alpha} m_{\alpha} y_{\alpha} z_{\alpha}$$

Note that the matrix is symmetric.

The moment of inertia around the principal rotational axes correspond to the eigenvalues of the matrix $\underline{\underline{I}}$.

Matrices and vectors are most conveniently implemented in Python as arrays from the [numpy](http://www.numpy.org) module. The eigenvalues can be calculated using the functions of the [numpy.linalg](https://docs.scipy.org/doc/numpy/reference/routines.linalg.html) sub-module.

In solving the exercise the values of the atomic masses are needed. Accompanying this notebook you will find the file [atomic_masses.py](atomic_masses.py) containing a dictionary called *mass_dictionary* and a list called *mass_list*, both containing the same atomic mass data in g.mol<sup>-1</sup> but in different format. It is simpler to obtain the atomic masses  for an element from the dictionary. For example, for carbon one would do:

    from atomic_masses import *
    mass_dictionary["C"]
    
If you don't feel comfortable using dictionaries, you can use the mass_list, where each element is of the form

    [element_symbol_string,atomic_mass]

(you can open the file atomic_masses.py on a text editor and see for yourself).

Accompanying this notebook is the file [hemolysin.xyz](hemolysin.xyz) containing the structure of the &alpha;-Hemolysin, a membrane protein with a large pore that can induce the lysis of red blood cells. Full marks will be awarded to any implementation that produces the correct result against the hemolysin.xyz file and a blind test case.

If your implementation is functional (i.e. your program does not crash) but does not fully work for a generic case, partial marks will be awarded for correct implementation of the functions below.

## Implementation suggestion

A working solution can be obtained by appropriately combining the following functions.

### parse_line(*string*) - 1 mark

This function should receive one string as its argument (a line of the XYZ file) and return a list of the form:

    [element_symbol,coordinate_list]

where element_symbol is a string with the chemical symbol letter(s), and coordinate_list is a list of numbers with the atom coordinates. For example:

    ['S',[66.52700,-0.24800,33.94300]]
    
### get_masses(*list*) - 1 mark

This function should receive one list containing strings with chemical symbols, and should return a 1D array with the atomic masses for each element in the original list.


### cm_position(*array1*,*array2*) - 1 mark

This function should take two arrays of numbers: array1 is a 1D array with mass data, and array2 is a 2D array where each row contains position data. The function should return a 1D array with the position of the centre of mass.

### buildi(*array*) - 1 mark

This function receives a 1D array with the coordinates of one atom

    array([x,y,z])
    
end returns a 3&times;3 array corresponding to a matrix of the form

$$\left[ \begin{array}{ccc} y^2+z^2 & -x y & -x z \\ -x y & x^2+z^2 & -y z \\ -x z & -y z & x^2+y^2 \end{array}\right]$$


### inertia_matrix(*array1*,*array2*) - 1 mark

This function should take two arrays of numbers: array1 is a 1D array with mass data, and array2 is a 2D array where each row contains position data. The function should return a 3&times;3 array corresponding to the inertia matrix.

## Instructions

You should submit to Blackboard one file called *moi.py* containing a standalone Python script with your program. Your program should not crash when given a well formatted XYZ file. Any program that does not fulfil this criteria will not be considered.

In the same zip archive you will find a template for *moi.py*. Separate the function definition from the rest of your program which you should write inside the *if* block provided.

If you choose to follow the suggested implementation, in order to qualify for partial marks, it is important that you define the functions name and argument order *exactly* as listed above. We will test that parse_line(), get_masses(), cm_position() , buildi() and inertia_matrix() are present and behave appropriately. Functions that work correctly but are called different names will not result in partial marks being awarded. For the functions to be tested they need to be imported via an *import* statement, so it is advisable that you check you are able to import you functions from the *moi.py* file.

It is good practice to test your code. Choose a small molecule where you can calculate the moment of inertial about the 3 axis by hand, build the corresponding XYZ file, and test your program.

If you encounter difficulties, you are invited to ask questions in the forum set up for the course on Blackboard.